In [1]:
import pickle


In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import torch
from openai import OpenAI
import torch.nn.functional as F
import pickle

# OpenAI 클라이언트 설정
client = OpenAI(
    api_key="사용자 API 키",
    base_url="https://api.upstage.ai/v1/solar"
)

# 데이터 로딩 (이 코드는 엑셀 기준)
df = pd.read_excel('사용자 데이터셋 경로')

# 결측값 처리
df['DATA'] = df['DATA'].fillna('')

# label이 1인 데이터 필터링 (csv로 쓸때는 사용X)
filtered_df = df[df['TARGET'] == 1]

# 문장과 메뉴 이름을 리스트로 변환
texts = filtered_df['DATA'].tolist()
menu_names = filtered_df['MENU_NAME'].tolist()  # MENU_NAME 컬럼 추가

# OpenAI 임베딩 생성 함수
def get_embeddings(texts, batch_size=100):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        response = client.embeddings.create(
            input=batch_texts,
            model="solar-embedding-1-large-query"
        )
        batch_embeddings = [np.array(embedding.embedding) for embedding in response.data]
        embeddings.extend(batch_embeddings)
    return embeddings

# 필터링된 데이터의 임베딩 생성
embeddings = get_embeddings(texts)

# 딕셔너리로 만들기
answer = {}
for idx in range(len(embeddings)) :
  key = menu_names[idx].strip()
  value = embeddings[idx]
  answer[key] = value

# 임베딩을 피클 파일로 저장
# Google Drive의 특정 경로에 'embeddings.pkl'로 저장
save_path = '특정경로'
with open(save_path, 'wb') as f:
    pickle.dump(embeddings, f)
